In [ ]:
import optuna
import pandas as pd
import numpy as np
import yaml
import pickle
import torch
import os

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import tensor, cat, save, load, optim, nn
from torch.utils.data import DataLoader

from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend

import src.study_handler as sh
from src.utils import print_yaml

In [ ]:
# ----------------------------
# Set device
# ----------------------------
sh.DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

In [ ]:
#-------------------#
#  Prepare dataset  #
#-------------------#
from src.dataset_handler import processDataset, loadDataset

config = None
with open("./study.yaml") as file:
    config = yaml.safe_load(file)
    
print_yaml(config)
    
data_cfg = config['data']

trainset, testset = loadDataset(data_cfg)
train_dataset, test_dataset, train_indices, test_indices = processDataset(data_cfg, trainset, testset)

# Assign the datasets
sh.train_dataset = train_dataset
sh.test_dataset = test_dataset

In [ ]:
# ------------------------#
#        Run study        #
# ------------------------#
import src.save_load as sl

def run_optimization(config):
    study_cfg = config['study']
    
    metadata = sl.buildStudyMetadata(study_cfg, config['data'])
    _, save_path = sl.saveStudy(metadata, savePath=config['study']['root'])
    
    journal_path = os.path.join(save_path, "journal.log")
    storage = JournalStorage(JournalFileBackend(file_path=journal_path))
    
    study = optuna.create_study(
        study_name=study_cfg["study_name"],
        storage=storage,
        load_if_exists=True,
        direction="maximize"
    )
    func = lambda trial: sh.objective(trial, config)
    
    study.optimize(func, n_trials=study_cfg["trials"])
    print(f"Study '{study_cfg['study_name']}' completed. Best value: {study.best_value}, params: {study.best_params}")
    
    df = study.trials_dataframe()
    df.to_csv(os.path.join(save_path, "results.csv"), index=False)
    print(f"📄 Results saved to {os.path.join(save_path, 'results.csv')}")

    return study

if config is not None:
    run_optimization(config)